# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# План выполнения:

- 1. Открытие файла с данными и его изучение
- 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
- 3. Исследование качества разных моделей с помощью изменения гиперпараметров. Краткое описание выводов.
- 4. Проверка качества модели на тестовой выборке.
- 5. Проверка моделей на вменяемость.

# Описание данных:

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Наименования столбцов:
- **сalls** — количество звонков,
- **minutes** — суммарная длительность звонков в минутах,
- **messages** — количество sms-сообщений,
- **mb_used** — израсходованный интернет-трафик в Мб,
- **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# Подключение библиотек:

In [3]:
import warnings; warnings.filterwarnings("ignore", category=Warning)
import pandas as pd

In [11]:
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# 1. Загрузка и изучение файла.

In [5]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## 1.1 Проведение дополнительной проверки по обработке данных.

Хотя в условии сказано что предобработку данных выполнять не требуется, для большей уверенности проведем небольшую проверку данных, **в которой нам необходимо будет проверить**:
- **Отсутствие отрицательных значений** во всех столбцах
- Действительно ли **столбец 'is_ultra' содержит только значения 0 и 1**
- **Равны ли дробные части значений в столбцах 'calls' и 'minutes' нулю**, так как у нас установлен тип float64:
    - *Если равны*, тогда просто приведем тип данных в этих столбцах к int64
    - *Если не равны*, тогда округлим значения и приведем тип данных к int64.
- Присутствуют ли **дубликаты** в нашем датасете


In [7]:
if (df < 0).values.any() == True:
    print('Присутствуют отрицательные значения в датасете.')
else:
    print('Отсутствуют отрицательные значения в датасете.')

print('')

if (all(df['is_ultra'].isin([0, 1]))) == True:
    print('Столбец "is_ultra" содержит только значения 0 и 1.')
else:
    print('Столбец "is_ultra" содержит иные значения, помимо 0 и 1.')

print('')

if df['calls'].mod(1).sum() == 0:
    print('Дробные части значений в столбце "calls" равны нулю.')
    df['calls'] = df['calls'].astype('int64')
    print('Столбец "calls" успешно приведен к типу -', df['calls'].dtype)
else:
    print('Дробные части значений в столбце "calls" неравны нулю.')

print('') 

if df['messages'].mod(1).sum() == 0:
    print('Дробные части значений в столбце "messages" равны нулю.')
    df['messages'] = df['messages'].astype('int64')
    print('Столбец "messages" успешно приведен к типу -', df['messages'].dtype)
else:
    print('Дробные части значений в столбце "messages" неравны нулю.')

print('')

if df.duplicated().sum() == 0:
    print('Дубликаты в датасете отсутствуют.')
else:
    print('Присутствуют дубликаты в датасете.')


Отсутствуют отрицательные значения в датасете.

Столбец "is_ultra" содержит только значения 0 и 1.

Дробные части значений в столбце "calls" равны нулю.
Столбец "calls" успешно приведен к типу - int64

Дробные части значений в столбце "messages" равны нулю.
Столбец "messages" успешно приведен к типу - int64

Дубликаты в датасете отсутствуют.


Наблюдаем, что действительно **нашему датасету не требуется предобработка данных**, за исключением приведения к нужному типу данных некоторых столбцов. Соответственно, можем смело приступать к следующему этапу.

# 2. Разделение данных на выборки.

**Для разбивки датасета на обучающую/валидационную/тестовую выборки**, мы выполним следующие действия:
- 1. Разобъем датасет на два датафрейма с признаками и целевым признаком (принадлежность к тарифной группе)
- 2. Напишем функцию, которая на входе будет получать два созданных выше датафрейма, и на выходе разделит их на три выборки: обучающую/валидационную/тестовую.
- 3. После этого, выведет на экран информацию о размере полученных выборок в % соотношении от исходного датасета, и долю абонентов "Ультра" в наших выборках, для проверки их сбалансированности.

In [9]:
random_state = 11111
x_par = df.drop(['is_ultra'], axis = 1)
y_tar = df['is_ultra']

def train_valid_test_splitter(x_par, y_tar):
    
    text_prints = ['обучающей', 'валидационной', 'тестовой', 
                   'исходном датасете', 'обучающей выборке', 'валидационной выборке', 'тестовой выборке']
    
    global x_train, x_valid, x_test, y_train, y_valid, y_test
    x_train, x_valid, y_train, y_valid = train_test_split(x_par, y_tar, test_size = 0.4, random_state=random_state)
    x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size = 0.5, random_state = random_state)
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_valid = scaler.transform(x_valid)
    x_test = scaler.transform(x_test)
    
    samples = [x_train, x_valid, x_test, y_tar, y_train, y_valid, y_test]
    
    for count in range (0, 7):
        if count < 3:
            print('Размер ' + text_prints[count] + ' выборки в % от исходного датасета =', 
                  round(len(samples[count]) / len(df), 2) * 100, '%')
        elif count >= 3:
            print('Доля абонентов "Ультра" в ' + text_prints[count], 
                  round(len(samples[count][samples[count] == 1]) / len(samples[count]), 2) * 100, '%')


In [12]:
train_valid_test_splitter(x_par, y_tar)

Размер обучающей выборки в % от исходного датасета = 60.0 %
Размер валидационной выборки в % от исходного датасета = 20.0 %
Размер тестовой выборки в % от исходного датасета = 20.0 %
Доля абонентов "Ультра" в исходном датасете 31.0 %
Доля абонентов "Ультра" в обучающей выборке 30.0 %
Доля абонентов "Ультра" в валидационной выборке 32.0 %
Доля абонентов "Ультра" в тестовой выборке 31.0 %


Видим, что **разбиение на выборки выполнено успешно.** Получили обучающую выборку в размере 60% от исходного датасета, и валидационную и тестовую выборки по 20%, соответственно, от исходного датасета.

Также наблюдаем что **выборки получились сбалансированные**, доли абонентов тарифа "Ультра" во всех выборках составляют 30-32%, что является допустимым интервалом, учитывая что в исходном датасете данная доля составляла 31%.

Теперь **приступим к выбору наиболее оптимальной модели**.

# 3. Исследование моделей.

Исследование моделей, будем проводить используя подбор гиперпараметров с помощью GridSearchCV. Сначала для каждой модели получим наиболее оптимальные гиперпараметры (включая проведение пятикратной кросс-валидации, и после этого сравним результаты каждой оптимальной модели друг с другом, для выбора наилучшей.

Для этого, будем использовать такие модели как:

- **Дерево решений** - DecisionTreeClassifier
- **Логистическая регрессия** - LogisticRegression
- **Случайный лес** - RandomForestClassifier
- **Метод ближайших соседей** - KNeighborsClassifier

## 3.1. Дерево решений.

Введем следующие интервалы параметров для их тестирования с помощью GridSearchCV:
- *max_depth* - от 1 до 20
- *min_samples_leaf* - от 1 до 10
- *min_samples_split* - от 2 до 10

После этого сразу же обучим модель, используя полученные оптимальные параметры, выведем их и точность нашей модели по обучающей и валидационной выборке на экран.

In [13]:
parametrs = {'max_depth': range (1, 20),
             'min_samples_leaf': range (1,10),
             'min_samples_split': range (2,10,)}

model = DecisionTreeClassifier(random_state = random_state)
grid = GridSearchCV(model, parametrs, cv = 5)
grid.fit(x_train, y_train)

tree_model = DecisionTreeClassifier(**grid.best_params_, random_state = random_state)
tree_model.fit(x_train, y_train)
tree_train_predictions = tree_model.predict(x_train)
tree_valid_predictions = tree_model.predict(x_valid)
tree_accuracy_train = (round(accuracy_score(y_train, tree_train_predictions), 3))
tree_accuracy_valid = (round(accuracy_score(y_valid, tree_valid_predictions), 3))

print('\033[1m' + 'Оптимальные параметры для модели, полученные с помощью GridSearchCV:' + '\033[0m')
print('Максимальная глубина дерева -', grid.best_params_['max_depth'])
print('Минимальное число объектов в листе -', grid.best_params_['min_samples_leaf'])
print('Минимальный размер сэмпла для сплита -', grid.best_params_['min_samples_split'])
print('Точность на обучающей выборке -', tree_accuracy_train)
print('Точность на валидационной выборке -', tree_accuracy_valid)

Оптимальные параметры для модели, полученные с помощью GridSearchCV:
Максимальная глубина дерева - 8
Минимальное число объектов в листе - 9
Минимальный размер сэмпла для сплита - 2
Точность на обучающей выборке - 0.846
Точность на валидационной выборке - 0.795


Полученная модель, построенная с помощью GridSearchCV, проходит по нашему порогу адекватности в 0.75. 
Переходим к следующей модели.

## 3.2. Логистическая регрессия.

In [14]:
parametrs = {'solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
             'intercept_scaling': [0.5, 1.0, 1.5, 2.0, ],
             'class_weight': [None, 'balanced'],
             'C': [0.5, 1, 1.5, 2]}

model = LogisticRegression(random_state = random_state)
grid = GridSearchCV(model, parametrs, scoring='accuracy', verbose=1, cv=5)
grid.fit(x_train, y_train)

logistic_model = LogisticRegression(**grid.best_params_, random_state = random_state)
logistic_model.fit(x_train, y_train)
logistic_train_predictions = logistic_model.predict(x_train)
logistic_valid_predictions = logistic_model.predict(x_valid)
logistic_accuracy_train = (round(accuracy_score(y_train, logistic_train_predictions), 3))
logistic_accuracy_valid = (round(accuracy_score(y_valid, logistic_valid_predictions), 3))

print('\033[1m' + 'Оптимальные параметры для модели, полученные с помощью GridSearchCV:' + '\033[0m')
print('Параметр регуляризации -', grid.best_params_['C'])
print('Веса классов -', grid.best_params_['class_weight'])
print('Масштаб точки пересечения -', grid.best_params_['intercept_scaling'])
print('Используемый решатель -', grid.best_params_['solver'])
print('Точность на обучающей выборке -', logistic_accuracy_train)
print('Точность на валидационной выборке -', logistic_accuracy_valid)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Оптимальные параметры для модели, полученные с помощью GridSearchCV:
Параметр регуляризации - 0.5
Веса классов - None
Масштаб точки пересечения - 0.5
Используемый решатель - lbfgs
Точность на обучающей выборке - 0.757
Точность на валидационной выборке - 0.74


[Parallel(n_jobs=1)]: Done 640 out of 640 | elapsed:    5.0s finished


Модель логистической регрессии с параметрами, полученными через GridSearchCV, едва проходит порог адекватности в 0.75 даже на обучающей выборке, на валидационной выборке модель не прошла порог.

## 3.3. Случайный лес.

In [15]:
parametrs = {'max_depth': range (1, 10),
             'min_samples_leaf': range (5,10),
             'min_samples_split': range (2,5),
             'n_estimators': range (1, 5)}

model = RandomForestClassifier(random_state = random_state)
grid = GridSearchCV(model, parametrs, cv = 5)
grid.fit(x_train, y_train)

forest_model = RandomForestClassifier(**grid.best_params_, random_state = random_state)
forest_model.fit(x_train, y_train)
forest_train_predictions = forest_model.predict(x_train)
forest_valid_predictions = forest_model.predict(x_valid)
forest_accuracy_train = (round(accuracy_score(y_train, forest_train_predictions), 3))
forest_accuracy_valid = (round(accuracy_score(y_valid, forest_valid_predictions), 3))

print('\033[1m' + 'Оптимальные параметры для модели, полученные с помощью GridSearchCV:' + '\033[0m')
print('Максимальная глубина дерева -', grid.best_params_['max_depth'])
print('Количество деревьев -', grid.best_params_['n_estimators'])
print('Минимальное число объектов в листе -', grid.best_params_['min_samples_leaf'])
print('Минимальный размер сэмпла для сплита -', grid.best_params_['min_samples_split'])
print('Точность на обучающей выборке -', forest_accuracy_train)
print('Точность на валидационной выборке -', forest_accuracy_valid)

Оптимальные параметры для модели, полученные с помощью GridSearchCV:
Максимальная глубина дерева - 6
Количество деревьев - 4
Минимальное число объектов в листе - 9
Минимальный размер сэмпла для сплита - 2
Точность на обучающей выборке - 0.822
Точность на валидационной выборке - 0.782


Модель случайного леса с параметрами GridSearchCV, на данный момент, показывает наилучший результат с точностью в 0.82 на обучающей выборке, и 0.782 на валидационной выборке.

## 3.4. Метод ближайших соседей.

In [16]:
parametrs = {'n_neighbors': range(1, 20),
              'weights': ['uniform', 'distance'],
              'algorithm': ['ball_tree', 'kd_tree', 'brute']}

model = KNeighborsClassifier()
grid = GridSearchCV(model, parametrs, cv = 5)
grid.fit(x_train, y_train)

neighbors_model = KNeighborsClassifier(**grid.best_params_)
neighbors_model.fit(x_train, y_train)
neighbors_train_predictions = neighbors_model.predict(x_train)
neighbors_valid_predictions = neighbors_model.predict(x_valid)
neighbors_accuracy_train = (round(accuracy_score(y_train, neighbors_train_predictions), 3))
neighbors_accuracy_valid = (round(accuracy_score(y_valid, neighbors_valid_predictions), 3))

print('\033[1m' + 'Оптимальные параметры для модели, полученные с помощью GridSearchCV:' + '\033[0m')
print('Количество соседей -', grid.best_params_['n_neighbors'])
print('Метод расчета веса -', grid.best_params_['weights'])
print('Алгоритм -', grid.best_params_['algorithm'])
print('Точность на обучающей выборке -', neighbors_accuracy_train)
print('Точность на валидационной выборке -', neighbors_accuracy_valid)

Оптимальные параметры для модели, полученные с помощью GridSearchCV:
Количество соседей - 16
Метод расчета веса - distance
Алгоритм - ball_tree
Точность на обучающей выборке - 1.0
Точность на валидационной выборке - 0.792


Модель, построенная с помощью метода ближайших соседей на параметрах от GridSearchCV показывает хороший результат на обучающей выборке с точностью в 0.793, однако результат сильно ухудшается на валидационной выборке, и модель едва проходит порог адекватности.

**Общие выводы по моделям:**
Проверив четыре модели - дерево решений, логистическая регрессия, случайный лес, метод ближайших соседей, мы получили следующие результаты на валидационных и тестовых выборках:
- **Дерево решений:**
    - Точность на обучающей выборке: 0.846
    - Точность на валидационной выборке: 0.793
    
    
- **Логистическая регрессия:**
    - Точность на обучающей выборке: 0.752
    - Точность на валидационной выборке: 0.737
    
    
- **Случайный лес:**
    - Точность на обучающей выборке: 0.822
    - Точность на валидационной выборке: 0.782
    
    
- **Метод ближайших соседей:**
    - Точность на обучающей выборке: 0.793
    - Точность на валидационной выборке: 0.753


- **Наилучший результат на валидационной выборке:** случайный лес с автоматическим подбором параметров с помощью GrisSearchCV.
    
Порог адекватности не прошла модель логистической регрессии, однако, так как её значение точности не намного меньше порога адекватности, мы включим эту модель для проверки на тестовой выборке.

# 4. Проверка моделей на тестовой выборке.

Для проверки моделей на тестовой выборке, напишем функцию, которая на входе будет получать список с полученными моделями, значениями точности на валидационных выборках, а также названия моделей. На выходе функция выведет на экран точность на тестовой выборке для каждой модели, разницу в точности между тестовой и валидационной выборкой, а также укажет на наилучший результат какой-либо модели.

In [12]:
models = [tree_model, logistic_model, forest_model, neighbors_model]
models_valid_accuracy = [tree_accuracy_valid, logistic_accuracy_valid, forest_accuracy_valid, neighbors_accuracy_valid]
model_names = ['Дерево решений',
              'Логистическая регрессия',
              'Случайный лес',
              'Метод ближайших соседей']

def model_testing(models, model_names):
    model_name = 0
    best_accuracy = 0
    for model in models:
        test_predictions = model.predict(x_test)
        accuracy = (round(accuracy_score(y_test, test_predictions), 3))
        print('\033[1m' + model_names[model_name] + '\033[0m')
        print('Точность на тестовой выборке =', accuracy)
        print('Разница в точности между тестовой и валидационной выборкой =', 
              round(accuracy - models_valid_accuracy[model_name], 3))
        print('')
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_names[model_name]
        model_name += 1
    print('\033[1m' + 'Наилучший результат показывает модель ' + "'" + best_model_name + "'" + ' с точностью -', best_accuracy)

In [13]:
model_testing(models, model_names)

Дерево решений
Точность на тестовой выборке = 0.776
Разница в точности между тестовой и валидационной выборкой = -0.017

Логистическая регрессия
Точность на тестовой выборке = 0.745
Разница в точности между тестовой и валидационной выборкой = 0.008

Случайный лес
Точность на тестовой выборке = 0.787
Разница в точности между тестовой и валидационной выборкой = 0.005

Метод ближайших соседей
Точность на тестовой выборке = 0.753
Разница в точности между тестовой и валидационной выборкой = 0.0

Наилучший результат показывает модель 'Случайный лес' с точностью - 0.787


С помощью полученной функции, мы видим, что **наилучший результат всё же показала модель "Случайный лес"** с результатом на тестовой выборке в 0.787, что является допустим значением для нашего порога адекватности.

**Модель логистической регрессии показала чуть более лучший результат** на тестовой выборке, однако всё же не проходит по нашему порогу адекватности, **занимая последнее место по точности.** 

**На третьем месте находится модель, построенная с помощью метода ближайших соседей**, с результатом в 0.753. 

**На втором месте находится модель дерева решений с точностью на тестовой выборке** в 0.776.